In [30]:
import requests
import pandas as pd
import numpy as np
import pickle
import os

# Set pandas options to display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)


In [31]:
kommune = [
    "Albertslund", "Allerød", "Assens", "Ballerup", "Billund", "Bornholms", "Brøndby", "Brønderslev-Dronninglund", "Dragør",
    "Egedal", "Esbjerg", "Fanø", "Favrskov", "Faxe", "Fredensborg", "Fredericia", "Frederiksberg", "Frederikshavn", "Frederikssund",
    "Frederiksværk-Hundested", "Furesø", "Faaborg-Midtfyn", "Gentofte", "Gladsaxe", "Glostrup", "Greve", "Gribskov", "Guldborgsund",
    "Haderslev", "Hedensted", "Helsingør", "Herlev", "Herning", "Hillerød", "Hjørring", "Holbæk", "Holstebro", "Horsens", "Hvidovre",
    "Høje-Taastrup", "Hørsholm", "Ikast-Brande", "Ishøj", "Jammerbugt", "Kalundborg", "Kerteminde", "Kolding", "København", "Køge",
    "Langeland", "Lejre", "Lemvig", "Lolland", "Lyngby-Tårbæk", "Læsø", "Mariagerfjord", "Middelfart", "Morsø", "Norddjurs", "Nordfyns",
    "Nyborg", "Næstved", "Odder", "Odense", "Odsherred", "Randers", "Rebild", "Ringkøbing-Skjern", "Ringsted", "Roskilde", "Rudersdal",
    "Rødovre", "Samsø", "Silkeborg", "Skanderborg", "Skive", "Slagelse", "Solrød", "Sorø", "Stevns", "Struer", "Svendborg", "Syddjurs",
    "Sønderborg", "Thisted", "Tønder", "Tårnby", "Vallensbæk", "Varde", "Vejen", "Vejle", "Vesthimmerlands", "Viborg", "Vordingborg", "Ærø",
    "Aabenraa", "Aalborg", 'Aarhus'
]

kode = [
    "0165", "0201", "0420", "0151", "0530", "0400", "0153", "0810", "0155", "0240", "0561", "0563", "0710", "0320", "0210", "0607",
    "0147", "0813", "0250", "0260", "0190", "0430", "0157", "0159", "0161", "0253", "0270", "0376", "0510", "0766", "0217", "0163",
    "0657", "0219", "0860", "0316", "0661", "0615", "0167", "0169", "0223", "0756", "0183", "0849", "0326", "0440", "0621", "0101",
    "0259", "0482", "0350", "0665", "0360", "0173", "0825", "0846", "0410", "0773", "0707", "0480", "0450", "0370", "0727", "0461",
    "0306", "0730", "0840", "0760", "0329", "0265", "0230", "0175", "0741", "0740", "0746", "0779", "0330", "0269", "0340", "0336",
    "0671", "0479", "0706", "0540", "0787", "0550", "0185", "0187", "0573", "0575", "0630", "0820", "0791", "0390", "0492", "0580",
    "0851", "0751"
]

kommune_df = pd.DataFrame({'kommune': kommune, 'kode': kode})

## DAGI

In [32]:
def get_DAGI(kommunekode):
    url = f'https://api.dataforsyningen.dk/adgangsadresser?kommunekode={kommunekode}&struktur=mini'
    response = requests.get(url)
    data = response.json()
    dagi_df = pd.DataFrame(data)
    return dagi_df

## BBR

In [33]:
def search_bbr(page=1, pagesize=1000, kommunekode='0825'):

    api_url = "https://services.datafordeler.dk/BBR/BBRPublic/1/rest/enhed"

    params = {
        "username": "ANLVUSSNAP",
        "password": "",
        "Format": "JSON",
        "Kommunekode": kommunekode,
        "pagesize": pagesize,  # Specify the maximum page size
        "page": page  # Request the first page
    }

    response = requests.get(api_url, params=params)
    data = response.json()
    bbr_df = pd.DataFrame(data)
    return bbr_df

def get_bbr(code='0825'):
    page = 1
    bbr_kommune = pd.DataFrame()

    while True:
        df = search_bbr(page=page, pagesize=1000, kommunekode=code)
        
        if bbr_kommune.empty:
            bbr_kommune = df
        else:
            bbr_kommune = pd.concat([bbr_kommune, df])
        
        if df.empty:
            break
        else:
            page += 1
            
    return bbr_kommune

In [ ]:
def process_data(path, get_data_func,amount=None):
    """
    Processes data from a given path using a specified data retrieval function.

    Parameters:
    path (str): The path to the pickle file where the DataFrame is stored.
    get_data_func (function): The function used to retrieve data. This function should take a code and return a DataFrame.

    Returns:
    all_data (DataFrame): The final DataFrame after all data has been retrieved and concatenated.
    """
    if os.path.exists(path):
        all_data = pd.read_pickle(path)
    else:
        all_data = pd.DataFrame()
        all_data.to_pickle(path)

    names = np.array(kommune_df['kommune'])
    codes = np.array(kommune_df['kode'])

    for index, code in enumerate(codes[:amount]):
        if all_data.empty:
            all_data = get_data_func(code)
            print(names[index])
        else:
            all_data = pd.concat([all_data, get_data_func(code)])
            print(names[index])
        
        print(f'{index+1} / {len(codes)}')    
        all_data.to_pickle(path)
        print('saved')

    return all_data

df = process_data('BBR.pkl', get_bbr, amount=2)
